In [ ]:
%pip install --upgrade geopy ipykernel matplotlib numpy openpyxl pandas pip plotly_express polars PyCap python-dotenv pyspark seaborn setuptools tabulate tabula-py
# %pip install pandas==1.5.3 [pandas_on_spark]
# %pip install distutils
# %pip install pyarrow
%pip list

In [ ]:
# Import necessaery packages
from pyspark.sql import SparkSession
from pyspark.sql import functions as spark_func
import pandas as pd
from itertools import chain

# Some settings
str_file = "fix_Peka40STR2023.txt"
address_cols = ["NoKPKIR", "Alamat", "Poskod", "Bandar", "Negeri", "state",]

# Create SparkSession
spark = SparkSession.builder.appName("ReplacePostalCode").getOrCreate()

# Read the csv file
df = spark.read.options(delimiter="|", header=True).csv(str_file)

# Select the important columns for address, then cast string to postcode, capitalize address, city, state, create a column of state
address_df = df.select(*address_cols[0:5])\
               .withColumn("Poskod", df["Poskod"].cast("string"))\
               .withColumn("state", spark_func.upper("Negeri"))
# address_df = address_df.withColumn("Alamat", spark_func.regexp_replace(address_df["Alamat"], "^#", ""))

# To capitalize all the address related information
for column in address_cols[1:]:
    address_df = address_df.withColumn(column, spark_func.upper(spark_func.trim(column)))
               
# Create list of state
# ['W.PERSEKUTUAN (KL)', 'JOHOR', 'KEDAH', 'PERAK', 'PERLIS', 'PULAU PINANG', 'W.PERSEKUTUAN (LABUAN)', 
#  'SELANGOR', 'TERENGGANU', 'NEGERI SEMBILAN', 'KELANTAN', 'SARAWAK', 'W.PERSEKUTUAN (PUTRAJAYA)', 'PAHANG', 'MELAKA', 'SABAH']
state_df = address_df.groupBy("Negeri").count()
state_list = [item["Negeri"] for item in state_df.sort("Negeri").toLocalIterator()]

# Create another state column by changing federal states' name
state_change_dict = {"W.PERSEKUTUAN (KL)":"KUALA LUMPUR",
                     "W.PERSEKUTUAN (LABUAN)":"LABUAN",
                     "W.PERSEKUTUAN (PUTRAJAYA)":"PUTRAJAYA"}

# Change the state name
for key in state_change_dict:
    address_df = address_df.withColumn("state", spark_func.when(spark_func.col("state") == key, state_change_dict[key])\
                                       .otherwise(spark_func.col("state")))

for item in reversed(address_cols[2:]):
    # To replace all poskod, city and state in the address
    address_df = address_df.withColumn("Alamat", spark_func.regexp_replace(address_df["Alamat"], address_df[item], ""))
    # To remove all those end with ,
    address_df = address_df.withColumn("Alamat", spark_func.regexp_replace(address_df["Alamat"], ",\\s*$", ""))
    address_df = address_df.withColumn("Alamat", spark_func.regexp_replace(address_df["Alamat"], ",\\s*", ", "))
    address_df = address_df.withColumn("Alamat", spark_func.regexp_replace(address_df["Alamat"], ",\\s*,\\s*", ", "))
    address_df = address_df.withColumn("Alamat", spark_func.regexp_replace(address_df["Alamat"], "\\s*,", ","))
    address_df = address_df.withColumn("Alamat", spark_func.trim(spark_func.regexp_replace(address_df["Alamat"], "\\s+", " ")))
    
address_df = address_df.withColumn("address", spark_func.when(spark_func.col("Alamat") != "", spark_func.concat_ws(" ", *address_cols[1:4], "state")))
address_df1 = address_df.filter(spark_func.col("address").isNotNull())
# address_df1.dropDuplicates(["address"])
address_df1.select(address_cols[0], "address").show(100, truncate = False)

In [ ]:
# for value in reversed(address_cols[2:]):
    # print(item)
    # expr(f"regexp_replace(Alamat, '{col_name}$', '')"))
    # spark_func.expr(spark_func.trim(address_df["Alamat"]), address_df[item], ""))
    # address_df = address_df.withColumn("Alamat", spark_func.regexp_replace(spark_func.trim(address_df["Alamat"]), f'\\b{item}\\b$', ''))
    # address_df = address_df.withColumn("Alamat", spark_func.expr(f"CASE WHEN split(Alamat, ' ')[-1] = '{value}' THEN regexp_replace(Alamat, ' {value}$', '') ELSE Alamat END"))

# df1 = address_df1.dropDuplicates(["address"]).select(address_cols[0], "address")
# df1.show(truncate = False)
address_df1.select(address_cols[0], "address")\
    .coalesce(1).write.format("csv").mode('overwrite').options(header='True', delimiter='|')\
        .save("str_address")

In [ ]:
def clean_address(df, columns_to_loop, column_need_to_be_clean,):
    for value in columns_to_loop:
        df = df.withColumn("words", spark_func.split(column_need_to_be_clean, "\\s+"))\
               .withColumn("last", spark_func.expr("words[size(words) - 1]"))\
               .withColumn("words", spark_func.expr("slice(words, 1, size(words) - 1)"))
        df = df.withColumn("last", spark_func.when(df[value]==df["last"], "").otherwise(df["last"]))
        df = df.withColumn(column_need_to_be_clean, spark_func.trim(spark_func.concat_ws(" ", df["words"], df["last"])))
        df = df.withColumn(column_need_to_be_clean, spark_func.regexp_replace(df[column_need_to_be_clean], ",\\s*$", ""))
        df = df.withColumn(column_need_to_be_clean, spark_func.regexp_replace(df[column_need_to_be_clean], ",\\s*", ", "))
        df = df.withColumn(column_need_to_be_clean, spark_func.regexp_replace(df[column_need_to_be_clean], ",\\s*,\\s*", ", "))
        df = df.withColumn(column_need_to_be_clean, spark_func.regexp_replace(df[column_need_to_be_clean], "\\s*,", ","))
        df = df.withColumn(column_need_to_be_clean, spark_func.trim(spark_func.regexp_replace(df[column_need_to_be_clean], "\\s+", " ")))
        return df.drop("words", "last")
clean_address(address_df, reversed(address_cols[2:]), "Alamat").show(truncate = False)

In [ ]:
# Import necessaery packages
from pyspark.sql import SparkSession
from pyspark.sql import functions as spark_func
import pandas as pd
from itertools import chain

# Some settings
str_file = "fix_Peka40STR2023.txt"
address_cols = ["NoKPKIR", "Alamat", "Poskod", "Bandar", "Negeri", "state",]

# Create SparkSession
spark = SparkSession.builder.appName("ReplacePostalCode").getOrCreate()

# Read the csv file
df = spark.read.options(delimiter="|", header=True).csv("str_address")
df = df.pandas_api()

In [ ]:
# with pd.ExcelWriter("str_address.xlsx", mode="w") as writer:
#     for num in range(0, len(df), 1000):
#         df.iloc[num:num+1000].to_excel(writer, sheet_name = f"sheet{num/1000}", index = False)

for num in range(0, len(df), 1000):
    df.iloc[num:num+1000].to_excel(f"str_partition/sheet{num/1000}.xlsx", sheet_name = f"sheet{num//1000}", index = False)

In [5]:
import pandas as pd
import tabula
file_path = "/Users/wh0102/Downloads/github/7046_mphd/MGT GROUPING 23.24.pdf"
df = tabula.read_pdf(file_path, pages = "all", lattice = False, stream = False)
df[1].to_clipboard(index = False)

/Users/wh0102/Downloads/github/.venv/lib/python3.12/site-packages/tabula/io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
